# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Narrowing Data to My Ideal Weather Conditions
My ideal weather conditions inculde
- Anywhere between 75 and 80 degrees.
- A pleasent breeze (5 - 24 mph)
- Some clouds in the sky to admire (20% to 50%)

In [2]:
#import current weather data from csv
clean_weather_data = pd.read_csv('../Output/clean_weather_data.csv')

In [3]:
# narrowing the data to my ideal weather conditions
ideal_weather_data = clean_weather_data.loc[(clean_weather_data['Temperature (F)'] >= 75) & (clean_weather_data['Temperature (F)'] < 80),:]
ideal_weather_data = ideal_weather_data.loc[(ideal_weather_data['Wind Speed (mph)'] >= 5) & (ideal_weather_data['Wind Speed (mph)'] < 24),:]
ideal_weather_data = ideal_weather_data.loc[(ideal_weather_data['Cloudiness (%)'] >= 20) & (ideal_weather_data['Cloudiness (%)'] < 50),:]
#ideal_weather_data

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [5]:
# Store 'Lat' and 'Lng' into  locations 
locations = ideal_weather_data[["Latitude", "Longitude"]]
humidity = ideal_weather_data['Humidity (%)']

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

NameError: name 'hotel_df' is not defined

In [ ]:
# Add marker layer ontop of heat map


# Display figure
